### 1. 패키지 설치

### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [1]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"로그인 상태입니다. 사용자: {user_info['name']}")
except Exception as e:
    print("로그인되지 않았거나 토큰이 유효하지 않습니다.")
    print(e)

/home/chae/faiss_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


로그인 상태입니다. 사용자: chaeeee


In [2]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [4]:
from langchain.schema import Document
import os, json
from tqdm import tqdm

# 1. JSON 파일 경로 설정
json_path = "./json_data.json"  # 단일 JSON 파일 경로

# 2. JSON 데이터 불러오기
with open(json_path, 'r', encoding='utf-8') as f:
    all_data = json.load(f)

# 3. {}를 기준으로 JSON 데이터 분할 및 Document 객체 생성
documents = []

for data in tqdm(all_data, desc="Generating Documents", unit="entry", ncols=80):
    metadata = {
        "title": data.get('title', 'N/A'),
        "artist": data.get('artist', 'N/A'),
        "year": data.get('year', 'N/A'),
        "read_count": data.get('read_count', 0)
    }

    # JSON 데이터의 각 항목을 Document 객체로 변환
    doc_content = json.dumps(data, ensure_ascii=False, indent=4)
    documents.append(Document(
        page_content=doc_content.strip(),
        metadata=metadata
    ))

Generating Documents: 100%|█████████| 11029/11029 [00:00<00:00, 63433.55entry/s]


In [5]:
len(documents)

11029

In [6]:
documents[1]

Document(metadata={'title': '팔괘호', 'artist': '한기석', 'year': '1960', 'read_count': 129}, page_content='{\n    "title": "팔괘호",\n    "title_ch": "八卦壺",\n    "title_eng": "Palgwae Vase",\n    "artist": "한기석",\n    "artist_eng": "HAN Kisuk",\n    "artwork_number": 2,\n    "year": "1960",\n    "size": "250×127",\n    "materials": "캔버스, 종이에 유화 물감",\n    "category": "회화 II",\n    "description": "한농(韓農) 한기석(1930-2011)은 표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는 항아리와 화병을 계속 그렸는데, 항아리에 대한 사람들의 일반적 향수 이미지는 그의 작품에서 친근감을 불어넣어 준다. 또한 완벽한 균형을 이루면서 노련하게 표현되어 다양한 색의 조화를 지닌 도자기의 미를 느낄 수 있다.",\n    "read_count": 129\n}')

In [10]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.schema import Document

# 텍스트 분할 설정
text_splitter = CharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=128  # 오버랩 설정
)

# 단일 DOCX 파일 로드
file_path = "./dataset2.docx"  # 파일 경로를 이곳에 입력하세요
loader = Docx2txtLoader(file_path)
raw_text = loader.load()[0].page_content  # DOCX 파일의 전체 텍스트 가져오기
print("raw_text 개수: ", len(raw_text))
      
# 작품명을 기준으로 텍스트 분리
def split_artwork_documents(doc_text):
    artworks = doc_text.split("\n\n작품명:")  # 작품을 구분
    documents = []

    for artwork in artworks:
        if artwork.strip():  # 빈 텍스트 제외
            # "작품명:" 추가로 일관성 유지
            doc_content = "작품명:" + artwork if not artwork.startswith("작품명:") else artwork

            # 메타데이터 초기화
            metadata = {}
            lines = doc_content.split("\n")  # 텍스트 줄 단위로 나누기

            # 메타데이터 추출
            for line in lines:
                if line.startswith("작품명:"):
                    metadata["작품명"] = line.replace("작품명:", "").strip()
                elif line.startswith("작가:"):
                    metadata["작가"] = line.replace("작가:", "").strip()
                elif line.startswith("제작 연도:"):
                    metadata["제작 연도"] = line.replace("제작 연도:", "").strip()
                elif line.startswith("카테고리:"):
                    metadata["카테고리"] = line.replace("카테고리:", "").strip()

            # Document 객체 생성
            documents.append(Document(
                page_content=doc_content.strip(),
                metadata=metadata
            ))

    return documents



# 작품별 Document 생성
documents = split_artwork_documents(raw_text)

# 생성된 작품별 Document에 대해 추가 청크 분할
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)
    for chunk in chunks:
        # 청크에 원래 Document의 메타데이터 유지
        chunked_documents.append(Document(
            page_content=chunk,
            metadata=doc.metadata  # 원본 메타데이터 복사
        ))

print(f"총 문서 수: {len(chunked_documents)}")

raw_text 개수:  8042447
총 문서 수: 15195


In [7]:
documents[11001]

Document(metadata={'title': '물방앗간의 아침', 'artist': '김기창 a', 'year': '1939', 'read_count': 30}, page_content='{\n    "title": "물방앗간의 아침",\n    "title_ch": "N/A",\n    "title_eng": "Watermill in the Morning",\n    "artist": "김기창 a",\n    "artist_eng": "KIM Kichang a",\n    "artwork_number": 11002,\n    "year": "1939",\n    "size": "130×162",\n    "materials": "비단에 색",\n    "category": "회화 I",\n    "description": "운보(雲甫) 김기창(金基昶, 1914-2001)은 어렸을 때 장티푸스에 걸려 청각장애를 갖게 되었다. 17세에 이당 김은호(1892-1979)의 문하에 들어가 동양화를 사사했다. 1930년대에는 《조선미술전람회》 꾸준히 작품을 출품하며 활발한 활동을 했다. 작가는 운포(雲圃)라는 호를 썼으나, 광복 후에는 운보(雲甫)라고 쓰기 시작했으며 일본화풍 청산을 위해 적극적인 모색의 시간을 보냈다. 이후 1947년에 우향 박래현(1920-1976)과 결혼하면서, 1947년부터 1972년까지 거의 매해 부부전을 개최했다. 한국전쟁 때 군산으로 피난 가 있는 동안 반추상과 입체주의를 도입한 새로운 양식의 동양화를 실험하면서 점차 추상과 구상을 자유롭게 넘나드는 폭넓은 창작 세계를 보여주었다. 1957년에 한국화의 새로운 방향을 모색하고자 하는 단체인 백양회(白陽會)를 결성했으며, 1961년에는 동남아 순회전을 하기도 했다. 1975년 즈음부터는 특유의 청록산수, 바보화풍을 제작했다.<물방앗간의 아침>은 물레방아가 돌아가는 방앗간의 아침 풍경을 그린 작품으로, 1939년 《제18회 조선미술전람회》에 입선했다.* 김기창의 초기작에 속하는 이 작품은 

In [8]:
# 특정 Document 객체의 텍스트 길이 확인
len(documents[11001].page_content)

1009

In [9]:
# 800자를 초과하는 Document 개수 세기
over_800_count = sum(1 for doc in documents if len(doc.page_content) > 800)

# 결과 출력
print(f"800자를 초과하는 Document 개수: {over_800_count}")


800자를 초과하는 Document 개수: 6460


In [10]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss  # FAISS 라이브러리 필요

# 1. 임베딩 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


In [15]:
# 2. 문서 데이터와 메타데이터 분리
texts = [doc.page_content for doc in documents]  # 문서 텍스트
metadatas = [doc.metadata for doc in documents]  # 문서 메타데이터

# 3. 문서 임베딩 생성
embeddings = embedding_model.encode(texts)

In [16]:
# 4. FAISS 인덱스 생성
embedding_dim = embeddings.shape[1]  # 벡터 차원 확인
faiss_index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 인덱스
faiss_index.add(embeddings)  # 벡터 추가


In [17]:
from langchain.docstore.in_memory import InMemoryDocstore

# 5. Docstore 생성
# 각 문서에 고유 ID를 부여해 InMemoryDocstore 생성
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})

In [18]:
# 6. FAISS 벡터스토어 생성
def embed_query(text):
    return embedding_model.encode([text])[0]  # 단일 쿼리 텍스트를 임베딩

index_to_docstore_id = {i: str(i) for i in range(len(documents))}

faiss_db = FAISS(
    embedding_function=embed_query,
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# 7. FAISS 데이터베이스 저장
faiss_db.save_local("./faiss_artworks_0114_docx")
print("FAISS 데이터베이스가 성공적으로 저장되었습니다!")


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 저장되었습니다!


In [20]:
# 9. 검색 테스트
query = "박승무의 설경"
results = faiss_db.similarity_search(query, k=5)

# 10. 검색 결과 출력
for result in results:
    print("문서 텍스트:", result.page_content)
    print("문서 메타데이터:", result.metadata)
    print('=------------------------------------------')


문서 텍스트: 작품명: 설경(雪景) / N/A / Snowy Landscape

작가: 박승무 / PARK Seungmoo

작품 번호: 9401

제작 연도: 1965

크기: 157×157

재료: 종이에 먹, 색

카테고리: 회화 I

작품 설명: 심향(深香) 박승무(朴勝武, 1893-1980)는 1913년부터 서화미술회의 안중식(安中植, 1861-1919)과 조석진(趙錫晋, 1853-1920) 문하에서 전통화법을 배웠으며, 졸업한 이듬해 중국 상하이로 향했으나 1919년 3·1 운동 이후 귀국했다. 《조선미술전람회》와 《서화협회전람회》에 출품하며 활동을 시작하였고, 광복 이후에는 대전에 정착하여 작업을 지속했다. 1957년 제1회 충청남도 문화상 미술부문 수상자로 선정되었다.<설경>은 눈 온 후의 산촌 풍경을 담은 산수화로, 박승무가 즐겨 그렸던 주제 중 하나이다. 작가는 화면 왼쪽에 커다란 산을 배치하고, 오른쪽으로는 공간을 시원하게 열어주는 구도를 취했다. 근경의 지팡이를 든 노인과 아이는 박승무의 산수화에 자주 등장하는 인물상으로, 점경인물과 가옥이 어우러져 정감있게 표현되었다.
문서 메타데이터: {'작품명': '설경(雪景) / N/A / Snowy Landscape', '작가': '박승무 / PARK Seungmoo', '제작 연도': '1965', '카테고리': '회화 I'}
=------------------------------------------
문서 텍스트: 작품명: 설경 / 雪景 / Snowscape

작가: 박승무 / PARK Seungmoo

작품 번호: 3371

제작 연도: 1974

크기: 65×255

재료: 종이에 먹, 색

카테고리: 회화 I

작품 설명: 심향(深香)박승무(1893-1980)는 설경을 위주로 작품을 해왔다. 눈 덮인 초가집과 나무들의 사실적인 표현과는 달리 배경을 이루는 산의 묘사는 구도에서 도식적이리 만큼 빈번히 등장한다. 먼 산의 윤곽선 및 나무들의 부드럽고 정감 짙은 풍정미, 잔 점과 선의 표현적

In [3]:
from langchain_community.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


# 기존 DB 로드 
persist_directory = "./faiss_artworks_0114_docx"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 로드되었습니다!


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [5]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:31<00:00,  4.49s/it]


In [6]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda
/tmp/ipykernel_6010/1315975019.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with:
"The document does not contain this information." in Korean.

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in **Korean**.
Always use **beautiful Markdown formatting** (headings, bullet points, bold or italic text) to enhance readability.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [53]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 20,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.8      # 다양성과 관련성 간 균형
    }
)


In [54]:
import re
class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        # <assistant> 이후의 텍스트만 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            # 마크다운 코드 블록으로 출력 포맷
            return f"### 모델 결과\n\n{extracted_text}\n\n"
        else:
            # <assistant> 태그가 없는 경우 원래 출력 반환
            return f"### 모델 결과\n\n{llm_output.strip()}\n\n"


In [55]:
from langchain.schema.runnable import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers.string import StrOutputParser
from langchain.prompts import ChatPromptTemplate
chain = (
    RunnableMap({
        "context": retriever,               # Retriever에서 반환된 값을 가져옴
        "question": RunnablePassthrough()   # 질문은 그대로 전달
    })
    | (lambda x: {
        "context": "\n".join([doc.page_content for doc in x["context"]]),
        "question": x["question"]
    })  # context를 문자열로 변환
    | prompt                               # Prompt Template에 전달
    | llm                                  # LLM으로 응답 생성
    | MarkdownOutputParser()                    # 응답을 문자열로 변환
)


In [56]:
query = "이중섭 작가가 만난 사람들은? 또 언제 만났지?"

In [57]:
response = chain.invoke({"question": query})
print(response)

### 모델 결과

이중섭 작가가 만난 중요한 인물 중 하나는 일본에서 만난 야마모토 마사코입니다. 동경의 분카학원 시절에 야마모토 마사코와 연애 관계를 맺었으며, 이 시기의 작품들에서는 두 사람의 친밀한 관계를 암시하는 환상적이고 초현실주의적인 이미지들이 표현되어 있습니다. 정확한 연도는 명시되어 있지 않으나, 주로 1930년대 후반부터 1940년대 초반까지의 시기로 추정됩니다. 이 외에도 그의 작품에서 가족 구성원들과도 깊은 관계를 맺고 있었지만, 구체적인 개인에 대한 만남의 시기는 작품 설명에서 명확히 제시되지 않았습니다. 이중섭의 삶과 작품은 가족, 특히 가족을 일본으로 보낸 후의 상황과 깊은 연관이 있었음을 알 수 있습니다.




In [44]:
retrieved_docs = retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")  # 문서의 실제 내용
    print(f"Metadata: {doc.metadata}")    # 메타데이터 (예: 출처, 페이지 등)
    print("-" * 50)


Document 1:
Content: {
    "title": "흔적 백 F-76",
    "title_ch": "痕迹 白 F-76",
    "title_eng": "Traces. White F-76",
    "artist": "박길웅",
    "artist_eng": "PARK Kiloung",
    "artwork_number": 1430,
    "year": "1970",
    "size": "70×47",
    "materials": "종이에 아크릴릭 물감, 수채 믈김",
    "category": "회화 II",
    "description": "박길웅(1940-1977)의 회화는 국내 수학시절, 미국 체류시절과 그 직후 그리고 그 이후의 3기로 나눌 수 있다.국내 수학시절에 박길웅은 향후의 회화에 있어 기초 개념이 되는 소(素), 추념(追念), 향(向), 유(流), 정(靜) 등을 개발하였으며《제18회 대한민국미술전람회》(1969) 대통령상을 수상한 <흔적 백 F-75>(1969)에서 이러한 규칙들을 성공적으로 구현했다는 평가를 받았다.미국 체류시절과 그 직후의 작업은 <낮과 밤>, <대화>(1975-1976) 연작에서와 같이 음과 양, 여와 남 등을 상징하는 두 개의 대립하는 색채 및 형상을 보여주는 구조, 즉 '이항대립(二項對立)' 구조로 대표된다.한편, 그가 타계하던 1977년 한 해에 걸쳐 시도하였던 <원초공간>에서는 끝이 불규칙한 굵고 검은 띠를 횡으로 반복하면서 띠 가운데 강한 원색의 원환들을 새기듯 그려 넣는 화면으로 전환하였다.",
    "read_count": 3
}
Metadata: {'title': '흔적 백 F-76', 'artist': '박길웅', 'year': '1970', 'read_count': 3}
--------------------------------------------------
Document 2:
Content: {
    "title": "원전 76-3",
    "title_ch": 

In [28]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 검색된 문서 수 출력
print(f"검색된 문서 수: {len(docs_and_scores)}")

# 각 문서의 파일명, 전체 내용, 유사도 점수 출력
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n문서 {i}:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")


검색된 문서 수: 5

문서 1:
  파일명: N/A
  유사도 점수: 0.9295
  전체 내용: {
    "title": "페인팅-78-1",
    "title_ch": "N/A",
    "title_eng": "Painting-78-1",
    "artist": "이강소",
    "artist_eng": "LEE Kangso",
    "artwork_number": 7871,
    "year": "1978",
    "size": "30분",
    "materials": "단채널 영상, 컬러, 무음",
    "category": "뉴미디어",
    "description": "1970년대에 설치, 퍼포먼스, 판화, 비디오 등 다양한 매체로 전위미술운동을 전개했던 이강소(李康昭, 1943- )는 《대구현대미술제》를 개최하는데 있어 큰 역할을 담당한 바 있다. 《대구현대미술제》는 회화와 조각뿐만 아니라 설치와 야외 이벤트를 수용하면서 미술의 영역을 확장해 나갔고, 70년대 말에는 영상 작업도 받아들이게 된다. 그때 이강소는 박현기, 김영진, 최병소 등과 영상을 촬영했는데, 당시 제작했던 것이 <페인팅-78-1>이다. 이 작품은 《제4회 대구현대미술제》(1978)에 출품되었다.<페인팅-78-1>에서 작가는 카메라 앞에 유리를 세워놓고, 그 유리에 물감을 칠했다. 그러면 그것은 마치 카메라 렌즈에 물감을 칠하는 것처럼 보인다. 회화에 관심이 많았던 이강소는 그림을 그리는 과정을 관객에게 보여주고 싶었다고 한다. 화가가 캔버스에 그림을 그리면, 관객은 캔버스에 물감이 칠해지는 것을, 즉 캔버스의 앞면을 보게 된다. 하지만 <페인팅-78-1>에서 관객은 캔버스 뒷면 방향에서 물감이 칠해지는 것을 볼 수 있다.",
    "read_count": 499
}

문서 2:
  파일명: N/A
  유사도 점수: 0.9498
  전체 내용: {
    "title": "원전 76-3",
    "title_ch": "N/A",
    "title_eng":